# Exploratory Data Analysis of VGChartz.com Video Game Sales Data

## Features to look into:
- Proximity of release date to holiday
- Day of week of release date
- price of console
- price of game
- whether developer is in top 5 or not
- location of developer
- number of years the game has been released
## Things to note/do
- make sure to remove data of games that were included with console ie. WII SPORTS

In [105]:
# if needed: pip install requests or conda install requests
import requests
from bs4 import BeautifulSoup
import re
from lxml import etree
import lxml.html as lh
import pandas as pd

In [7]:
# Ping one page of search results in VGChartz.com
url = 'http://www.vgchartz.com/games/games.php?page=1&results=200&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No'

response = requests.get(url)

### Check the Status
response.status_code # status code = 200 => OK

200

In [59]:
#print(response.text)

In [10]:
page=response.text
soup = BeautifulSoup(page, "lxml")

In [107]:
#Store the contents of the website under doc
doc = lh.fromstring(response.content)

In [110]:
[len(T) for T in tr_elements[:15]]

[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [302]:
# Selects the table with all the data in it on HTML using xpath
tr_elements = doc.xpath('//*[@id="generalBody"]/table')[0]
tr_elements.find('a')
# Not sure how to find links

In [303]:

len(tr_elements.xpath("./tr"))

203

In [243]:
row_dict={}
df=pd.DataFrame()
row_list= list()
for counter,row in enumerate(tr_elements.xpath(".//tr")):
    if counter > 2:
        row_list=[td.text for td in row.xpath(".//td")]
        
        row_dict[counter] = row_list
# Test finding elements

df=pd.DataFrame.from_dict(row_dict).transpose()
df.columns = ['position','game','blank','console','publisher','developer','vgchart_score',\
             'critic_score','user_score','total_shipped','total_sales',\
              'na_sales','pal_sales','japan_sales','other_sales',\
              'release_date','last_update']

In [296]:
consoles = list()
#for table in soup.find_all('table'):
for img in soup.find_all('img'):
    if 'images/consoles'in img['src']:
        console_tag = (img['src'][17:-6])
        consoles.append(img['alt'])


In [300]:
df=df.reset_index().drop(columns = 'index')
df['console'] = consoles

In [301]:
df.head()

,position,game,blank,console,publisher,developer,vgchart_score,critic_score,user_score,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date,last_update
0,1,\n,,Wii,Nintendo,Nintendo EAD,N/A,7.7,N/A,82.86m,N/A,N/A,N/A,N/A,N/A,19th Nov 06,N/A
1,2,\n,,NES,Nintendo,Nintendo EAD,N/A,10.0,N/A,40.24m,N/A,N/A,N/A,N/A,N/A,18th Oct 85,N/A
2,3,\n,,Wii,Nintendo,Nintendo EAD,8.7,8.2,9.1,37.14m,N/A,N/A,N/A,N/A,N/A,27th Apr 08,11th Apr 18
3,4,\n,,PC,PUBG Corporation,PUBG Corporation,N/A,N/A,N/A,36.60m,N/A,N/A,N/A,N/A,N/A,21st Dec 17,13th Nov 18
4,5,\n,,Wii,Nintendo,Nintendo EAD,8.8,8.0,8.8,33.09m,N/A,N/A,N/A,N/A,N/A,26th Jul 09,N/A


In [86]:
# Use pandas to read the table in easily, but with multiple indexes. This lets us grab the name of the game\
# because it's stored as a link
pandas_shortcut = pd.read_html(url,header=None)

In [238]:
# Select row with name of games
games = pandas_shortcut[6]

In [236]:
# Takes pandas read_html dataframe and steals its name column without multiindex
df['game']=(games.iloc[:,2].to_list())

In [237]:
df


,position,game,blank,console,publisher,developer,vgchart_score,critic_score,user_score,total_shipped,total_sales,na_sales,pal_sales,japan_sales,other_sales,release_date,last_update
0,1,Wii Sports,,\n,Nintendo,Nintendo EAD,N/A,7.7,N/A,82.86m,N/A,N/A,N/A,N/A,N/A,19th Nov 06,N/A
1,2,Super Mario Bros.,,\n,Nintendo,Nintendo EAD,N/A,10.0,N/A,40.24m,N/A,N/A,N/A,N/A,N/A,18th Oct 85,N/A
2,3,Mario Kart Wii Read the review,,\n,Nintendo,Nintendo EAD,8.7,8.2,9.1,37.14m,N/A,N/A,N/A,N/A,N/A,27th Apr 08,11th Apr 18
3,4,PlayerUnknown's Battlegrounds,,\n,PUBG Corporation,PUBG Corporation,N/A,N/A,N/A,36.60m,N/A,N/A,N/A,N/A,N/A,21st Dec 17,13th Nov 18
4,5,Wii Sports Resort Read the review,,\n,Nintendo,Nintendo EAD,8.8,8.0,8.8,33.09m,N/A,N/A,N/A,N/A,N/A,26th Jul 09,N/A
5,6,Pokémon Red / Green / Blue Version,,\n,Nintendo,Game Freak,N/A,9.4,N/A,31.38m,N/A,N/A,N/A,N/A,N/A,30th Sep 98,N/A
6,7,New Super Mario Bros.,,\n,Nintendo,Nintendo EAD,N/A,9.1,8.1,30.80m,N/A,N/A,N/A,N/A,N/A,15th May 06,N/A
7,8,Tetris,,\n,Nintendo,Bullet Proof Software,N/A,N/A,N/A,30.26m,N/A,N/A,N/A,N/A,N/A,01st Jun 89,N/A
8,9,New Super Mario Bros. Wii Read the review,,\n,Nintendo,Nintendo EAD,9.1,8.6,9.2,30.22m,N/A,N/A,N/A,N/A,N/A,15th Nov 09,N/A
9,10,Minecraft,,\n,Mojang,Mojang AB,N/A,10.0,N/A,30.01m,N/A,N/A,N/A,N/A,N/A,10th May 10,05th Aug 18


In [240]:
test=5

In [241]:
test

5